In [15]:
import numpy as np

with open("C:\\Users\\HP\\Midnight_Project\data\\fer2013.csv") as f:
    content = f.readlines()

    lines = np.array(content)

    num_of_instances = lines.size
    print("number of instances: ",num_of_instances)

number of instances:  35888


In [16]:
import keras
import keras.utils
from keras.utils import np_utils

x_train, y_train, x_test, y_test = [], [], [], []

for i in range(1,num_of_instances):
    emotion, img, usage = lines[i].split(",")

    val = img.split(" ")
    pixels = np.array(val, 'float32')

    emotion = keras.utils.np_utils.to_categorical(emotion, 7)

    if 'Training' in usage:
        y_train.append(emotion)
        x_train.append(pixels)
    elif 'PublicTest' in usage:
        y_test.append(emotion)
        x_test.append(pixels)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D,AveragePooling2D

model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

In [18]:
np.shape(x_train), np.shape(y_train)

((28709, 2304), (28709, 7))

In [14]:
import tensorflow as tf

x_train = np.expand_dims(x_train, axis=3)
y_train = np.expand_dims(y_train, axis=3)

gen = tf.keras.preprocessing.image.ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=500)

model.compile(loss='categorical_crossentropy'
            , optimizer=tf.keras.optimizers.Adam()
            , metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=500, epochs=20)

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (28709, 2304, 1, 1, 1))

In [18]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])

test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.